# BMS_SOC_LSTM_1.2.0
- nicht mehr autoregressiv
- test funktioniert noch nicht

In [ ]:
from pathlib import Path
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
# Falls vorhanden:
from pytorch_forecasting.models.nn.rnn import LSTM as ForecastingLSTM

torch.cuda.empty_cache()

def load_cell_data(data_dir: Path):
    """
    Lädt df.parquet aus dem Unterordner 'MGFarm_18650_C01'.
    """
    dataframes = {}
    folder = data_dir / "MGFarm_18650_C01"
    if folder.exists() and folder.is_dir():
        df_path = folder / 'df.parquet'
        if df_path.exists():
            df = pd.read_parquet(df_path)
            dataframes["C01"] = df
            print(f"Loaded {folder.name}")
        else:
            print(f"Warning: No df.parquet found in {folder.name}")
    else:
        print("Warning: Folder MGFarm_18650_C01 not found")
    return dataframes


def main():
    ########################################################################
    # 1) Daten laden und vorbereiten
    ########################################################################
    data_dir = Path('/home/florianr/MG_Farm/5_Data/MGFarm_18650_Dataframes')
    cell_data = load_cell_data(data_dir)

    # Nimm die erste gefundene Zelle
    cell_keys = sorted(cell_data.keys())[:1]
    if len(cell_keys) < 1:
        raise ValueError("Keine Zelle gefunden; bitte prüfen.")

    cell_name = cell_keys[0]
    df_full = cell_data[cell_name]

    # Reduziere auf 25% der Daten (kannst du anpassen)
    sample_size = int(len(df_full) * 0.25)
    df_small = df_full.head(sample_size).copy()

    print(f"Gesamtdaten: {len(df_full)}, wir nehmen 25% = {sample_size} Zeilen.")

    # Timestamp anlegen (nur für Plot)
    df_small['timestamp'] = pd.to_datetime(df_small['Absolute_Time[yyyy-mm-dd hh:mm:ss]'])

    # Zeitbasierter Split: 40% Train, 40% Val, 20% Test
    len_small = len(df_small)
    train_end = int(len_small * 0.4)
    val_end   = int(len_small * 0.8)

    df_train = df_small.iloc[:train_end]
    df_val   = df_small.iloc[train_end:val_end]
    df_test  = df_small.iloc[val_end:]

    print(f"Train: {len(df_train)}  |  Val: {len(df_val)}  |  Test: {len(df_test)}")

    ########################################################################
    # 2) Skalierung von Voltage & Current (NICHT SOC)
    ########################################################################
    scaler = StandardScaler()
    features_to_scale = ['Voltage[V]', 'Current[A]']

    # Fit nur auf Training
    scaler.fit(df_train[features_to_scale])

    df_train_scaled = df_train.copy()
    df_val_scaled   = df_val.copy()
    df_test_scaled  = df_test.copy()

    df_train_scaled[features_to_scale] = scaler.transform(df_train_scaled[features_to_scale])
    df_val_scaled[features_to_scale]   = scaler.transform(df_val_scaled[features_to_scale])
    df_test_scaled[features_to_scale]  = scaler.transform(df_test_scaled[features_to_scale])

    ########################################################################
    # 3) Dataset-Klasse (seq2one, NICHT autoregressiv)
    ########################################################################
    class SequenceDataset(Dataset):
        """
        - X[t] = [Voltage, Current] für t..t+seq_len-1
        - y[t] = SOC an t+seq_len
        """
        def __init__(self, df, seq_len=60):
            self.seq_len = seq_len
            # Nur Voltage und Current als Features
            self.features = df[["Voltage[V]", "Current[A]"]].values
            # SOC als Label
            self.labels = df["SOC_ZHU"].values

        def __len__(self):
            return len(self.labels) - self.seq_len

        def __getitem__(self, idx):
            x_seq = self.features[idx : idx + self.seq_len]   # shape (seq_len, 2)
            y_val = self.labels[idx + self.seq_len]           # SOC-Wert
            return torch.tensor(x_seq, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)

    # Erstellen der Datasets
    seq_length = 60
    train_dataset = SequenceDataset(df_train_scaled, seq_len=seq_length)
    val_dataset   = SequenceDataset(df_val_scaled,   seq_len=seq_length)
    test_dataset  = SequenceDataset(df_test_scaled,  seq_len=seq_length)

    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=50000, shuffle=True, drop_last=True)
    val_loader   = DataLoader(val_dataset,   batch_size=50000, shuffle=False, drop_last=True)
    test_loader  = DataLoader(test_dataset,  batch_size=50000, shuffle=False, drop_last=True)

    ########################################################################
    # 4) LSTM-Modell + (optional) DirectionLoss
    ########################################################################
    # Falls du pytorch_forecasting nicht hast, kannst du stattdessen nn.LSTM nehmen.
    class LSTMSOCModel(nn.Module):
        def __init__(self, input_size=2, hidden_size=32, num_layers=2, batch_first=True):
            super().__init__()
            self.lstm = ForecastingLSTM(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=batch_first
            )
            self.fc = nn.Linear(hidden_size, 1)

        def forward(self, x):
            lstm_out, _ = self.lstm(x)
            last_out = lstm_out[:, -1, :]
            soc_pred = self.fc(last_out)
            return soc_pred.squeeze(-1)

    class DirectionLoss(nn.Module):
        """
        MSE + Strafterm für "falsche" Drehrichtung.
        """
        def __init__(self, alpha=0.1):
            super().__init__()
            self.mse = nn.MSELoss()
            self.alpha = alpha

        def forward(self, y_pred, y_true, x_seq):
            # 1) Standard MSE
            loss_mse = self.mse(y_pred, y_true)

            # 2) Strafterm (optional)
            current  = x_seq[:, -1, 1]  # Letzter Current aus dem Fenster
            soc_last = y_true           # Wir tun so, als wäre der "letzte SOC" der ground truth

            penalty_up   = torch.clamp(soc_last - y_pred, min=0.0) * (current > 0).float()
            penalty_down = torch.clamp(y_pred - soc_last, min=0.0) * (current < 0).float()
            penalty_direction = (penalty_up + penalty_down).mean()

            total_loss = loss_mse + self.alpha * penalty_direction
            return total_loss

    model = LSTMSOCModel(input_size=2, hidden_size=32, num_layers=2, batch_first=True)

    # Standard-Loss oder DirectionLoss
    criterion = DirectionLoss(alpha=0.1)
    # criterion = nn.MSELoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

    ########################################################################
    # 5) Training mit Validation + Early Stopping
    ########################################################################
    # Plot: Datenaufteilung
    plt.figure(figsize=(12,6))
    plt.plot(df_small['timestamp'], df_small['SOC_ZHU'], 'k-', label='SOC (alle Daten)')
    plt.axvspan(df_train['timestamp'].iloc[0],
                df_train['timestamp'].iloc[-1],
                color='green', alpha=0.3, label='Training')
    plt.axvspan(df_val['timestamp'].iloc[0],
                df_val['timestamp'].iloc[-1],
                color='orange', alpha=0.3, label='Validation')
    plt.axvspan(df_test['timestamp'].iloc[0],
                df_test['timestamp'].iloc[-1],
                color='red', alpha=0.3, label='Test')
    plt.xlabel('Time')
    plt.ylabel('SOC (ZHU)')
    plt.title('Datenaufteilung')
    plt.legend()
    plt.tight_layout()
    plt.show()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    epochs = 50
    patience = 10
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    # Gradient Accumulation
    accumulation_steps = 10

    for epoch in range(1, epochs+1):
        model.train()
        train_losses = []
        optimizer.zero_grad()

        for i, (x_batch, y_batch) in enumerate(train_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            y_pred = model(x_batch)
            loss = criterion(y_pred, y_batch, x_batch) / accumulation_steps
            loss.backward()

            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            train_losses.append(loss.item() * accumulation_steps)

        mean_train_loss = np.mean(train_losses)

        # Validation
        model.eval()
        val_losses = []
        all_y_val = []
        all_y_pred = []
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                y_pred_val = model(x_val)
                v_loss = criterion(y_pred_val, y_val, x_val)
                val_losses.append(v_loss.item())
                # ersten Batch zum Plot
                if not all_y_val:
                    all_y_val.append(y_val.cpu().numpy())
                    all_y_pred.append(y_pred_val.cpu().numpy())

        mean_val_loss = np.mean(val_losses)

        # Early Stopping
        if mean_val_loss < best_val_loss:
            best_val_loss = mean_val_loss
            best_model_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch} because val_loss not improved.")
                break

        print(f"Epoch {epoch:03d}/{epochs}, "
              f"Train Loss: {mean_train_loss:.6f}, "
              f"Val Loss: {mean_val_loss:.6f}, "
              f"NoImprove: {epochs_no_improve}")

        # Plot Validation Predictions (nur erster Batch)
        y_val_example = all_y_val[0].flatten()
        y_pred_example = all_y_pred[0].flatten()
        plt.figure(figsize=(10,4))
        plt.plot(y_val_example, label='Ground Truth')
        plt.plot(y_pred_example, label='Predicted')
        plt.title(f"Validation Predictions at Epoch {epoch}")
        plt.xlabel("Sample Index")
        plt.ylabel("SOC")
        plt.legend()
        plt.tight_layout()
        plt.show(block=False)
        plt.pause(1)
        plt.close()

    # Bestes Modell wiederherstellen
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"\nBest model reloaded with val_loss = {best_val_loss:.6f}")

    # Speichern
    models_dir = Path("models")
    os.makedirs(models_dir, exist_ok=True)
    
    model_path = models_dir / "best_lstm_soc_model.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Bestes Modell gespeichert unter: {model_path}")

    ########################################################################
    # 6) Test-Vorhersage (NICHT autoregressiv)
    ########################################################################
    # Lade dasselbe Modell nochmal
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    test_predictions = []
    test_targets = []

    with torch.no_grad():
        for x_test, y_test in test_loader:
            x_test, y_test = x_test.to(device), y_test.to(device)
            y_pred_test = model(x_test)
            test_predictions.append(y_pred_test.cpu().numpy())
            test_targets.append(y_test.cpu().numpy())

    test_predictions = np.concatenate(test_predictions)
    test_targets = np.concatenate(test_targets)

    # Zeitstempel anpassen
    time_test = df_test['timestamp'].values[seq_length:seq_length + len(test_targets)]

    plt.figure(figsize=(12,5))
    plt.plot(time_test, test_targets, label="Ground Truth SOC", linestyle='-')
    plt.plot(time_test, test_predictions, label="Predicted SOC (LSTM)", linestyle='--')
    plt.title(f"Standard LSTM-Vorhersage - Test (Zelle: {cell_name})")
    plt.xlabel("Time")
    plt.ylabel("SOC (ZHU)")
    plt.legend()
    plt.tight_layout()

    plot_file = models_dir / "prediction_test.png"
    plt.savefig(plot_file)
    plt.show()
    print(f"Test-Plot gespeichert unter: {plot_file}")


if __name__ == "__main__":
    main()


# BMS_SOC_LSTM_2.0.0
- nicht mehr autoregressiv
- test funktioniert noch nicht wird hier getestet !!

In [ ]:
from pathlib import Path
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
# Falls vorhanden:
from pytorch_forecasting.models.nn.rnn import LSTM as ForecastingLSTM

torch.cuda.empty_cache()

def load_cell_data(data_dir: Path):
    """
    Lädt df.parquet aus dem Unterordner 'MGFarm_18650_C01'.
    """
    dataframes = {}
    folder = data_dir / "MGFarm_18650_C01"
    if folder.exists() and folder.is_dir():
        df_path = folder / 'df.parquet'
        if df_path.exists():
            df = pd.read_parquet(df_path)
            dataframes["C01"] = df
            print(f"Loaded {folder.name}")
        else:
            print(f"Warning: No df.parquet found in {folder.name}")
    else:
        print("Warning: Folder MGFarm_18650_C01 not found")
    return dataframes


def main():
    ########################################################################
    # 1) Daten laden und vorbereiten
    ########################################################################
    data_dir = Path('/home/florianr/MG_Farm/5_Data/MGFarm_18650_Dataframes')
    cell_data = load_cell_data(data_dir)

    # Nimm die erste gefundene Zelle
    cell_keys = sorted(cell_data.keys())[:1]
    if len(cell_keys) < 1:
        raise ValueError("Keine Zelle gefunden; bitte prüfen.")

    cell_name = cell_keys[0]
    df_full = cell_data[cell_name]

    # Reduziere auf 25% der Daten (kannst du anpassen)
    sample_size = int(len(df_full) * 0.25)
    df_small = df_full.head(sample_size).copy()

    print(f"Gesamtdaten: {len(df_full)}, wir nehmen 25% = {sample_size} Zeilen.")

    # Timestamp anlegen (nur für Plot)
    df_small['timestamp'] = pd.to_datetime(df_small['Absolute_Time[yyyy-mm-dd hh:mm:ss]'])

    # Zeitbasierter Split: 40% Train, 40% Val, 20% Test
    len_small = len(df_small)
    train_end = int(len_small * 0.4)
    val_end   = int(len_small * 0.8)

    df_train = df_small.iloc[:train_end]
    df_val   = df_small.iloc[train_end:val_end]
    df_test  = df_small.iloc[val_end:]

    print(f"Train: {len(df_train)}  |  Val: {len(df_val)}  |  Test: {len(df_test)}")

    ########################################################################
    # 2) Skalierung von Voltage & Current (NICHT SOC)
    ########################################################################
    scaler = StandardScaler()
    features_to_scale = ['Voltage[V]', 'Current[A]']

    # Fit nur auf Training
    scaler.fit(df_train[features_to_scale])

    df_train_scaled = df_train.copy()
    df_val_scaled   = df_val.copy()
    df_test_scaled  = df_test.copy()

    df_train_scaled[features_to_scale] = scaler.transform(df_train_scaled[features_to_scale])
    df_val_scaled[features_to_scale]   = scaler.transform(df_val_scaled[features_to_scale])
    df_test_scaled[features_to_scale]  = scaler.transform(df_test_scaled[features_to_scale])

    ########################################################################
    # 3) Dataset-Klasse (seq2one, NICHT autoregressiv)
    ########################################################################
    class SequenceDataset(Dataset):
        """
        - X[t] = [Voltage, Current] für t..t+seq_len-1
        - y[t] = SOC an t+seq_len
        """
        def __init__(self, df, seq_len=60):
            self.seq_len = seq_len
            # Nur Voltage und Current als Features
            self.features = df[["Voltage[V]", "Current[A]"]].values
            # SOC als Label
            self.labels = df["SOC_ZHU"].values

        def __len__(self):
            return len(self.labels) - self.seq_len

        def __getitem__(self, idx):
            x_seq = self.features[idx : idx + self.seq_len]   # shape (seq_len, 2)
            y_val = self.labels[idx + self.seq_len]           # SOC-Wert
            return torch.tensor(x_seq, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)

    # Erstellen der Datasets
    seq_length = 60
    train_dataset = SequenceDataset(df_train_scaled, seq_len=seq_length)
    val_dataset   = SequenceDataset(df_val_scaled,   seq_len=seq_length)
    test_dataset  = SequenceDataset(df_test_scaled,  seq_len=seq_length)

    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=50000, shuffle=True, drop_last=True)
    val_loader   = DataLoader(val_dataset,   batch_size=50000, shuffle=False, drop_last=True)
    test_loader  = DataLoader(test_dataset,  batch_size=50000, shuffle=False, drop_last=True)

    ########################################################################
    # 4) LSTM-Modell + (optional) DirectionLoss
    ########################################################################
    # Falls du pytorch_forecasting nicht hast, kannst du stattdessen nn.LSTM nehmen.
    class LSTMSOCModel(nn.Module):
        def __init__(self, input_size=2, hidden_size=32, num_layers=2, batch_first=True):
            super().__init__()
            self.lstm = ForecastingLSTM(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=batch_first
            )
            self.fc = nn.Linear(hidden_size, 1)

        def forward(self, x):
            lstm_out, _ = self.lstm(x)
            last_out = lstm_out[:, -1, :]
            soc_pred = self.fc(last_out)
            return soc_pred.squeeze(-1)

    class DirectionLoss(nn.Module):
        """
        MSE + Strafterm für "falsche" Drehrichtung.
        """
        def __init__(self, alpha=0.1):
            super().__init__()
            self.mse = nn.MSELoss()
            self.alpha = alpha

        def forward(self, y_pred, y_true, x_seq):
            # 1) Standard MSE
            loss_mse = self.mse(y_pred, y_true)

            # 2) Strafterm (optional)
            current  = x_seq[:, -1, 1]  # Letzter Current aus dem Fenster
            soc_last = y_true           # Wir tun so, als wäre der "letzte SOC" der ground truth

            penalty_up   = torch.clamp(soc_last - y_pred, min=0.0) * (current > 0).float()
            penalty_down = torch.clamp(y_pred - soc_last, min=0.0) * (current < 0).float()
            penalty_direction = (penalty_up + penalty_down).mean()

            total_loss = loss_mse + self.alpha * penalty_direction
            return total_loss

    model = LSTMSOCModel(input_size=2, hidden_size=32, num_layers=2, batch_first=True)

    # Standard-Loss oder DirectionLoss
    criterion = DirectionLoss(alpha=0.1)
    # criterion = nn.MSELoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

    ########################################################################
    # 5) Training mit Validation + Early Stopping
    ########################################################################
    # Plot: Datenaufteilung
    plt.figure(figsize=(12,6))
    plt.plot(df_small['timestamp'], df_small['SOC_ZHU'], 'k-', label='SOC (alle Daten)')
    plt.axvspan(df_train['timestamp'].iloc[0],
                df_train['timestamp'].iloc[-1],
                color='green', alpha=0.3, label='Training')
    plt.axvspan(df_val['timestamp'].iloc[0],
                df_val['timestamp'].iloc[-1],
                color='orange', alpha=0.3, label='Validation')
    plt.axvspan(df_test['timestamp'].iloc[0],
                df_test['timestamp'].iloc[-1],
                color='red', alpha=0.3, label='Test')
    plt.xlabel('Time')
    plt.ylabel('SOC (ZHU)')
    plt.title('Datenaufteilung')
    plt.legend()
    plt.tight_layout()
    plt.show()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    epochs = 50
    patience = 10
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    # Gradient Accumulation
    accumulation_steps = 10

    for epoch in range(1, epochs+1):
        model.train()
        train_losses = []
        optimizer.zero_grad()

        for i, (x_batch, y_batch) in enumerate(train_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            y_pred = model(x_batch)
            loss = criterion(y_pred, y_batch, x_batch) / accumulation_steps
            loss.backward()

            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            train_losses.append(loss.item() * accumulation_steps)

        mean_train_loss = np.mean(train_losses)

        # Validation
        model.eval()
        val_losses = []
        all_y_val = []
        all_y_pred = []
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                y_pred_val = model(x_val)
                v_loss = criterion(y_pred_val, y_val, x_val)
                val_losses.append(v_loss.item())
                # ersten Batch zum Plot
                if not all_y_val:
                    all_y_val.append(y_val.cpu().numpy())
                    all_y_pred.append(y_pred_val.cpu().numpy())

        mean_val_loss = np.mean(val_losses)

        # Early Stopping
        if mean_val_loss < best_val_loss:
            best_val_loss = mean_val_loss
            best_model_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch} because val_loss not improved.")
                break

        print(f"Epoch {epoch:03d}/{epochs}, "
              f"Train Loss: {mean_train_loss:.6f}, "
              f"Val Loss: {mean_val_loss:.6f}, "
              f"NoImprove: {epochs_no_improve}")

        # Plot Validation Predictions (nur erster Batch)
        y_val_example = all_y_val[0].flatten()
        y_pred_example = all_y_pred[0].flatten()
        plt.figure(figsize=(10,4))
        plt.plot(y_val_example, label='Ground Truth')
        plt.plot(y_pred_example, label='Predicted')
        plt.title(f"Validation Predictions at Epoch {epoch}")
        plt.xlabel("Sample Index")
        plt.ylabel("SOC")
        plt.legend()
        plt.tight_layout()
        plt.show(block=False)
        plt.pause(1)
        plt.close()

    # Bestes Modell wiederherstellen
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"\nBest model reloaded with val_loss = {best_val_loss:.6f}")

    # Speichern
    models_dir = Path("models")
    os.makedirs(models_dir, exist_ok=True)
    
    model_path = models_dir / "best_lstm_soc_model.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Bestes Modell gespeichert unter: {model_path}")

    ########################################################################
    # 6) Test-Vorhersage (NICHT autoregressiv)
    ########################################################################
    # Lade dasselbe Modell nochmal
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    test_predictions = []
    test_targets = []

    with torch.no_grad():
        for x_test, y_test in test_loader:
            x_test, y_test = x_test.to(device), y_test.to(device)
            y_pred_test = model(x_test)
            test_predictions.append(y_pred_test.cpu().numpy())
            test_targets.append(y_test.cpu().numpy())

    test_predictions = np.concatenate(test_predictions)
    test_targets = np.concatenate(test_targets)

    # Zeitstempel anpassen
    time_test = df_test['timestamp'].values[seq_length:seq_length + len(test_targets)]

    plt.figure(figsize=(12,5))
    plt.plot(time_test, test_targets, label="Ground Truth SOC", linestyle='-')
    plt.plot(time_test, test_predictions, label="Predicted SOC (LSTM)", linestyle='--')
    plt.title(f"Standard LSTM-Vorhersage - Test (Zelle: {cell_name})")
    plt.xlabel("Time")
    plt.ylabel("SOC (ZHU)")
    plt.legend()
    plt.tight_layout()

    plot_file = models_dir / "prediction_test.png"
    plt.savefig(plot_file)
    plt.show()
    print(f"Test-Plot gespeichert unter: {plot_file}")


if __name__ == "__main__":
    main()
